# Electricity Price Explanation using Bagging and AdaBoost Models  : Ensemble Learning Project


This notebook details the steps to develop Bagging and AdaBoost models to facilitate electricity price explanation using input variables from the training dataset and predicting the same using the test dataset. It also includes the data pre-processing steps already covered in the "Data Cleaning" notebook.

Group Members: Abhimanyu Bhadauria, Aditi Agrawal, Saptarshi Banerjee, Shubham Bhandari

## Library import:

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
#import required libraries
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from itertools import product
from sklearn import preprocessing
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV

## Data Import:

In [38]:
# After downloading the X_train/X_test/Y_train .csv files in your working directory:

X_train = pd.read_csv('/content/x_train.csv')
Y_train = pd.read_csv('/content/y_train.csv')
X_test = pd.read_csv('/content/x_test.csv')
Y_test = pd.read_csv('/content/y_test.csv')

## Data Cleaning and Pre-processing:

In [39]:
#concat the X_train and Y_train
X_train = pd.concat([X_train, Y_train], axis=1)
X_test = pd.concat([X_test, Y_test], axis=1)


In [40]:
#sort the data in ascending order of DAY_ID
X_train = X_train.sort_values(by='DAY_ID')
X_train = X_train.reset_index(drop=True)

X_test = X_test.sort_values(by='DAY_ID')
X_test = X_test.reset_index(drop=True)


In [41]:
#split the X_train data to X_train_FR and X_train_DE based on the COUNTRY column
X_train_FR = X_train[X_train['COUNTRY'] == 'FR']
X_train_DE = X_train[X_train['COUNTRY'] == 'DE']

X_test_FR = X_test[X_test['COUNTRY'] == 'FR']
X_test_DE = X_test[X_test['COUNTRY'] == 'DE']

In [42]:
fill=['DE_RAIN','FR_RAIN','DE_WIND','FR_WIND','DE_TEMP','FR_TEMP']

In [43]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

def fill_missing_values(df, columns_to_fill):
    # Loop over the columns to fill
    for column in columns_to_fill:
        # Get the data for this column
        data = df[column]

        # Find where the NaNs are
        mask = np.isnan(data)

        # Prepare the data for ARIMA
        train_data = data[~mask]
        test_data = data[mask]

        # Fit the ARIMA model
        model = ARIMA(train_data, order=(5,1,0))
        model_fit = model.fit()

        # Use the model to predict the missing values
        predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1)

        # Fill the missing values with the predictions
        data[mask] = predictions

        # Use forward fill for any remaining missing values
        data = data.fillna(method='ffill')

        # Use backward fill for any remaining missing values
        data = data.fillna(method='bfill')

        # Update the column in the DataFrame
        df[column] = data
    return df

In [44]:
fill_missing_values(X_train_FR, fill)
fill_missing_values(X_train_DE, fill)

fill_missing_values(X_test_FR, fill)
fill_missing_values(X_test_DE, fill)

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,ID,TARGET
1,410,4,DE,0.944117,0.203547,0.372743,-0.372743,0.656335,0.519380,-0.656335,...,-0.359430,-0.614570,-0.734964,1.192221,0.701597,-0.357747,0.696316,-0.426880,410,1.103289
3,562,6,DE,0.120788,-0.744840,-1.195940,1.195940,-1.670709,1.164429,1.670709,...,0.149032,0.324280,1.119383,-0.288148,-0.894852,2.213681,0.352223,0.518026,562,1.110984
5,750,9,DE,1.056973,0.535641,0.989874,-0.989874,0.091679,-1.072153,-0.091679,...,-0.647654,0.666003,1.848971,0.007103,-0.182161,0.021228,0.484992,0.202597,750,0.462893
7,802,13,DE,-0.138870,-0.702079,-0.521880,0.521880,-1.067974,-0.670168,1.067974,...,-0.139418,0.573454,0.498127,-0.571953,-0.349293,-0.548502,0.578269,-0.831399,802,-0.360199
11,504,24,DE,0.143362,-0.524899,-1.030414,1.030414,-1.005184,0.832404,1.005184,...,-0.139418,0.573454,0.498127,-0.571953,-0.349293,0.232096,-0.078845,0.190155,504,-1.463765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,558,1197,DE,0.554256,-0.696734,-0.110097,0.110097,-0.302746,0.511653,0.302746,...,2.819208,0.388530,1.683905,2.226844,1.120592,0.000000,0.885600,1.313050,558,0.618505
645,510,1203,DE,1.036507,0.450521,1.088059,-1.088059,0.211113,-1.404554,-0.211113,...,-0.077136,0.991121,2.040713,-1.868996,-1.839327,1.003526,-0.603416,-0.143810,510,0.699819
647,139,1206,DE,0.611063,-0.166570,-0.221738,0.221738,0.045492,0.302293,-0.045492,...,1.041395,-0.763306,-0.341595,0.523483,0.117366,-0.234023,1.145696,0.262949,139,0.355194
650,452,1210,DE,1.741160,1.823518,1.588180,-1.588180,2.061425,0.083979,-2.061425,...,1.041395,-0.763306,-0.341595,0.523483,0.117366,-0.035345,1.760113,1.481091,452,-0.838942


In [45]:
columns_to_interpolate = ['DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT', 'DE_NET_IMPORT', 'FR_NET_IMPORT']
#Interpolates missing values in specific columns of the given dataframes.
X_train_FR[columns_to_interpolate] = X_train_FR[columns_to_interpolate].interpolate(method='linear', limit_direction='both', axis=0)
X_train_DE[columns_to_interpolate] = X_train_DE[columns_to_interpolate].interpolate(method='linear', limit_direction='both', axis=0)

X_test_FR[columns_to_interpolate] = X_test_FR[columns_to_interpolate].interpolate(method='linear', limit_direction='both', axis=0)
X_test_DE[columns_to_interpolate] = X_test_DE[columns_to_interpolate].interpolate(method='linear', limit_direction='both', axis=0)

In [46]:
def calculate_fuel_cost(df):
    """
    Calculates the fuel cost for different countries and fuel types.

    Parameters:
    df (pandas.DataFrame): The input DataFrame containing the necessary columns.

    Returns:
    pandas.DataFrame: The DataFrame with additional columns for fuel costs.

    """
    for ctry, kind in product(['DE', 'FR'], ['COAL', 'GAS']):
        df[f'{ctry}_{kind}_COST'] = df[f'{kind}_RET'] * df[f'{ctry}_{kind}']
    df['DE_LIGNITE_COST'] = df['COAL_RET'] * df['DE_LIGNITE']
    return df

In [47]:
X_train_DE = calculate_fuel_cost(X_train_DE)

X_test_DE = calculate_fuel_cost(X_test_DE)


In [48]:
X_train_FR = calculate_fuel_cost(X_train_FR)

X_test_FR = calculate_fuel_cost(X_test_FR)


In [49]:
carbon_footprint_weights = {'LIGNITE': 1.05, 'COAL': 0.85, 'GAS': 0.7}

In [50]:
def calculate_carbon_footprint(df, carbon_footprint_weights):
    """
    Calculates the carbon footprint for each country and fuel type based on the given dataframe and carbon footprint weights.
    """
    for ctry, kind in product(['DE', 'FR'], ['COAL', 'GAS']):
        df[f'{ctry}_{kind}_CARBON'] = carbon_footprint_weights[kind] * df['CARBON_RET'] * df[f'{ctry}_{kind}']
    df['DE_LIGNITE_CARBON'] = carbon_footprint_weights['LIGNITE'] * df['CARBON_RET'] * df['DE_LIGNITE']
    return df

In [51]:
X_train_FR = calculate_carbon_footprint(X_train_FR, carbon_footprint_weights)

X_test_FR = calculate_carbon_footprint(X_test_FR, carbon_footprint_weights)


In [52]:
X_train_DE = calculate_carbon_footprint(X_train_DE, carbon_footprint_weights)

X_test_DE = calculate_carbon_footprint(X_test_DE, carbon_footprint_weights)


In [53]:
fr_coal_on = -0.725

In [54]:
X_train_FR['FR_COAL_ON'] = 1
X_train_FR.loc[X_train_FR['FR_COAL'] < fr_coal_on, 'FR_COAL_ON'] = 0

X_test_FR['FR_COAL_ON'] = 1
X_test_FR.loc[X_test_FR['FR_COAL'] < fr_coal_on, 'FR_COAL_ON'] = 0

In [55]:
def country_flow(data):
	"""
	Transforms the given data by rearranging columns based on the country.
	"""
	df = data.copy()
	df['EXCHANGE'] = np.where(df['COUNTRY'] == 'DE', df['DE_FR_EXCHANGE'], df['FR_DE_EXCHANGE'])
	df['SELF_EXPORT'] = np.where(df['COUNTRY'] == 'DE', df['DE_NET_EXPORT'], df['FR_NET_EXPORT'])
	df['OTHER_EXPORT'] = np.where(df['COUNTRY'] == 'DE', df['FR_NET_EXPORT'], df['DE_NET_EXPORT'])
	df['SELF_CONSUMPTION'] = np.where(df['COUNTRY'] == 'DE', df['DE_CONSUMPTION'], df['FR_CONSUMPTION'])
	df['OTHER_CONSUMPTION'] = np.where(df['COUNTRY'] == 'DE', df['FR_CONSUMPTION'], df['DE_CONSUMPTION'])
	df = df.drop(['DE_CONSUMPTION', 'FR_CONSUMPTION', 'DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT',
				  'DE_NET_IMPORT', 'FR_NET_EXPORT', 'FR_NET_IMPORT'], axis=1)
	return df

In [56]:
X_train_FR = country_flow(X_train_FR)
X_train_DE = country_flow(X_train_DE)

X_test_FR = country_flow(X_test_FR)
X_test_DE = country_flow(X_test_DE)


In [5]:
Y_train_DE = X_train_DE[['TARGET']]
X_train_DE = X_train_DE.drop(['TARGET', 'ID', 'DAY_ID', 'COUNTRY'], axis=1)

Y_train_FR = X_train_FR[['TARGET']]
X_train_FR = X_train_FR.drop(['TARGET', 'ID', 'DAY_ID', 'COUNTRY'], axis=1)


In [6]:
Y_test_DE = X_test_DE[['TARGET']]
X_test_DE = X_test_DE.drop(['TARGET', 'ID', 'DAY_ID', 'COUNTRY'], axis=1)

Y_test_FR = X_test_FR[['TARGET']]
X_test_FR = X_test_FR.drop(['TARGET', 'ID', 'DAY_ID', 'COUNTRY'], axis=1)


In [ ]:
X_train_DE = preprocessing.normalize(X_train_DE)
X_test_DE = preprocessing.normalize(X_test_DE)

X_train_FR = preprocessing.normalize(X_train_FR)
X_test_FR = preprocessing.normalize(X_test_FR)

Y_train_DE = preprocessing.normalize(Y_train_DE)
Y_test_DE = preprocessing.normalize(Y_test_DE)

Y_train_FR = preprocessing.normalize(Y_train_FR)
Y_test_FR = preprocessing.normalize(Y_test_FR)

## Model Development:

**Bagging Model**

In [79]:
# Initialize the Bagging Regressor model
bagging_model_DE = BaggingRegressor(random_state=42)
bagging_model_FR = BaggingRegressor(random_state=42)

# Fit the model with the training data
bagging_model_DE.fit(X_train_DE, Y_train_DE.ravel())
bagging_model_FR.fit(X_train_FR, Y_train_FR.ravel())

# Predict the target variable for the test data
Y_pred_DE = bagging_model_DE.predict(X_test_DE)
Y_pred_FR = bagging_model_FR.predict(X_test_FR)


# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Compute the Mean Absolute Percentage Error (MAPE) of the predicted target variable against the test data
mape_de = mean_absolute_percentage_error(Y_test_DE.ravel(), Y_pred_DE)
mape_fr = mean_absolute_percentage_error(Y_test_FR.ravel(), Y_pred_FR)

# Compute the Mean Squared Error (MSE) and Mean Absolute Error (MAE) of the predicted target variable against the test data
mse_de = mean_squared_error(Y_test_DE, Y_pred_DE)
mae_de = mean_absolute_error(Y_test_DE, Y_pred_DE)

mse_fr = mean_squared_error(Y_test_FR, Y_pred_FR)
mae_fr = mean_absolute_error(Y_test_FR, Y_pred_FR)

print("Germany - MSE: ", mse_de)
print("Germany - MAE: ", mae_de)
print("Germany - MAPE: ", mape_de,"%")
print("France - MSE: ", mse_fr)
print("France - MAE: ", mae_fr)
print("France - MAPE: ", mape_fr,"%")

Germany - MSE:  1.1364359861591695
Germany - MAE:  0.957439446366782
Germany - MAPE:  95.74394463667821 %
France - MSE:  1.1942739726027396
France - MAE:  1.0210958904109588
France - MAPE:  102.10958904109589 %


**Bagging Model (with hyperparameter tuning and cross-validation)**

In [12]:
# Define the parameters for the grid search
parameters = {
    'n_estimators': [10, 50, 100, 200],
    'max_samples': [0.5, 0.75, 1.0],
    'max_features': [0.5, 0.75, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}

# Initialize the Bagging Regressor model
bagging_model_DE = BaggingRegressor(random_state=42)
bagging_model_FR = BaggingRegressor(random_state=42)

# Initialize the GridSearchCV model
grid_search_DE = GridSearchCV(bagging_model_DE, parameters, cv=5)
grid_search_FR = GridSearchCV(bagging_model_FR, parameters, cv=5)

# Fit the model with the training data
grid_search_DE.fit(X_train_DE, Y_train_DE.ravel())
grid_search_FR.fit(X_train_FR, Y_train_FR.ravel())

# Get the best parameters
best_params_DE = grid_search_DE.best_params_
best_params_FR = grid_search_FR.best_params_

# Print the best parameters
print("Best parameters for Germany: ", best_params_DE)
print("Best parameters for France: ", best_params_FR)

# Predict the target variable for the test data
Y_pred_DE = grid_search_DE.predict(X_test_DE)
Y_pred_FR = grid_search_FR.predict(X_test_FR)

# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Compute the Mean Absolute Percentage Error (MAPE) of the predicted target variable against the test data
mape_de = mean_absolute_percentage_error(Y_test_DE.ravel(), Y_pred_DE)
mape_fr = mean_absolute_percentage_error(Y_test_FR.ravel(), Y_pred_FR)

# Compute the Mean Squared Error (MSE) and Mean Absolute Error (MAE) of the predicted target variable against the test data
mse_de = mean_squared_error(Y_test_DE, Y_pred_DE)
mae_de = mean_absolute_error(Y_test_DE, Y_pred_DE)

mse_fr = mean_squared_error(Y_test_FR, Y_pred_FR)
mae_fr = mean_absolute_error(Y_test_FR, Y_pred_FR)

print("Germany - MSE: ", mse_de)
print("Germany - MAE: ", mae_de)
print("Germany - MAPE: ", mape_de,"%")
print("France - MSE: ", mse_fr)
print("France - MAE: ", mae_fr)
print("France - MAPE: ", mape_fr,"%")

Best parameters for Germany:  {'bootstrap': False, 'bootstrap_features': False, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 200}
Best parameters for France:  {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 100}
Germany - MSE:  1.0765463667820068
Germany - MAE:  0.9847058823529413
Germany - MAPE:  98.47058823529413 %
France - MSE:  1.0791446575342467
France - MAE:  1.0141917808219176
France - MAPE:  101.41917808219176 %


**AdaBoost Model**

In [81]:
# Initialize the AdaBoost Regressor model
boosting_model_DE = AdaBoostRegressor(random_state=42)
boosting_model_FR = AdaBoostRegressor(random_state=42)

# Fit the model with the training data
boosting_model_DE.fit(X_train_DE, Y_train_DE.ravel())
boosting_model_FR.fit(X_train_FR, Y_train_FR.ravel())

# Predict the target variable for the test data
Y_pred_DE = boosting_model_DE.predict(X_test_DE)
Y_pred_FR = boosting_model_FR.predict(X_test_FR)


# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Compute the Mean Absolute Percentage Error (MAPE) of the predicted target variable against the test data
mape_de = mean_absolute_percentage_error(Y_test_DE.ravel(), Y_pred_DE)
mape_fr = mean_absolute_percentage_error(Y_test_FR.ravel(), Y_pred_FR)

# Compute the Mean Squared Error (MSE) and Mean Absolute Error (MAE) of the predicted target variable against the test data
mse_de = mean_squared_error(Y_test_DE, Y_pred_DE)
mae_de = mean_absolute_error(Y_test_DE, Y_pred_DE)

mse_fr = mean_squared_error(Y_test_FR, Y_pred_FR)
mae_fr = mean_absolute_error(Y_test_FR, Y_pred_FR)

print("Germany - MSE: ", mse_de)
print("Germany - MAE: ", mae_de)
print("Germany - MAPE: ", mape_de,"%")
print("France - MSE: ", mse_fr)
print("France - MAE: ", mae_fr)
print("France - MAPE: ", mape_fr,"%")

Germany - MSE:  1.1031523068528057
Germany - MAE:  1.0125743147524482
Germany - MAPE:  101.25743147524481 %
France - MSE:  1.154529026874893
France - MAE:  1.0001919337390008
France - MAPE:  100.01919337390008 %


**AdaBoost Model (with hyperparameter tuning and cross-validation)**

In [14]:
# Define the parameters for the grid search
parameters = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.05, 0.1, 0.5, 1.0],
    'loss': ['linear', 'square', 'exponential']
}

# Initialize the AdaBoost Regressor model
boosting_model_DE = AdaBoostRegressor(random_state=42)
boosting_model_FR = AdaBoostRegressor(random_state=42)

# Initialize the GridSearchCV model
grid_search_DE = GridSearchCV(boosting_model_DE, parameters, cv=5)
grid_search_FR = GridSearchCV(boosting_model_FR, parameters, cv=5)

# Fit the model with the training data
grid_search_DE.fit(X_train_DE, Y_train_DE.ravel())
grid_search_FR.fit(X_train_FR, Y_train_FR.ravel())

# Get the best parameters
best_params_DE = grid_search_DE.best_params_
best_params_FR = grid_search_FR.best_params_

# Print the best parameters
print("Best parameters for Germany: ", best_params_DE)
print("Best parameters for France: ", best_params_FR)

# Predict the target variable for the test data
Y_pred_DE = grid_search_DE.predict(X_test_DE)
Y_pred_FR = grid_search_FR.predict(X_test_FR)

# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Compute the Mean Absolute Percentage Error (MAPE) of the predicted target variable against the test data
mape_de = mean_absolute_percentage_error(Y_test_DE.ravel(), Y_pred_DE)
mape_fr = mean_absolute_percentage_error(Y_test_FR.ravel(), Y_pred_FR)

# Compute the Mean Squared Error (MSE) and Mean Absolute Error (MAE) of the predicted target variable against the test data
mse_de = mean_squared_error(Y_test_DE, Y_pred_DE)
mae_de = mean_absolute_error(Y_test_DE, Y_pred_DE)

mse_fr = mean_squared_error(Y_test_FR, Y_pred_FR)
mae_fr = mean_absolute_error(Y_test_FR, Y_pred_FR)

print("Germany - MSE: ", mse_de)
print("Germany - MAE: ", mae_de)
print("Germany - MAPE: ", mape_de,"%")
print("France - MSE: ", mse_fr)
print("France - MAE: ", mae_fr)
print("France - MAPE: ", mape_fr,"%")

Best parameters for Germany:  {'learning_rate': 0.1, 'loss': 'linear', 'n_estimators': 50}
Best parameters for France:  {'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 50}
Germany - MSE:  1.0708428143582807
Germany - MAE:  0.9885108570057505
Germany - MAPE:  98.85108570057504 %
France - MSE:  1.0411528761073419
France - MAE:  1.009280733427297
France - MAPE:  100.9280733427297 %
